In [ ]:
!pip install Flask pyngrok faiss-cpu numpy pymongo sentence-transformers requests huggingface-hub



In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import faiss
import numpy as np
from pymongo import MongoClient
from sentence_transformers import SentenceTransformer
import requests
from huggingface_hub import login

# Authenticate Hugging Face
login("")

# Step 1: Connect to MongoDB Atlas
print("Connecting to MongoDB Atlas...")
client = MongoClient("mongodb+srv://@cluster0.v70wf.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0")# ICI MET LE CODE ET PASSWROD DE MONGODB  ATLAS
db = client["AI"]
collection = db["AIPROJECT"]

print("Loading dataset from MongoDB...")
documents = []
for doc in collection.find():
    if "name" in doc and "instructions" in doc:
        combined_text = (
            f"Exercise Name: {doc['name']}. "
            f"Force: {doc.get('force', 'N/A')}. "
            f"Level: {doc.get('level', 'N/A')}. "
            f"Mechanic: {doc.get('mechanic', 'N/A')}. "
            f"Equipment: {doc.get('equipment', 'N/A')}. "
            f"Primary Muscles: {', '.join(doc.get('primaryMuscles', []))}. "
            f"Secondary Muscles: {', '.join(doc.get('secondaryMuscles', []))}. "
            f"Category: {doc.get('category', 'N/A')}. "
            f"Instructions: {' '.join(doc['instructions'])}."
        )
        documents.append(combined_text)
print(f"Loaded {len(documents)} documents from MongoDB.")

# Step 2: Initialize SentenceTransformer and Encode Dataset
print("Encoding documents with SentenceTransformer...")
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = embed_model.encode(documents)

# Step 3: Build FAISS Index
print("Building FAISS index...")
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))
print(f"FAISS index built with {index.ntotal} documents.")

# Step 4: Define Retrieval Function
def retrieve(query, top_k=7):
    query_embedding = embed_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)

    retrieved_docs = []
    for i in indices[0]:
        doc = documents[i]
        formatted_doc = (
            f"Exercise Name: {doc.split('.')[0].split(': ')[1]}\n"
            f"Equipment: {doc.split('Equipment: ')[1].split('.')[0]}\n"
            f"Primary Muscles: {doc.split('Primary Muscles: ')[1].split('.')[0]}\n"
            f"Secondary Muscles: {doc.split('Primary Muscles: ')[1].split('.')[0]}\n"
            f"Instructions: {doc.split('Instructions: ')[1]}"
        )
        retrieved_docs.append(formatted_doc)

    print("\n--- DEBUG: Retrieved Contexts ---")
    for doc in retrieved_docs:
        print(doc)

    return "\n\n".join(retrieved_docs)



Connecting to MongoDB Atlas...
Loading dataset from MongoDB...
Loaded 873 documents from MongoDB.
Encoding documents with SentenceTransformer...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Building FAISS index...
FAISS index built with 873 documents.


In [ ]:
# Step 5: Define Gemini API Call
def generate_with_gemini(context, query, api_key):
    endpoint = "https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent"
    headers = {"Content-Type": "application/json"}
    payload = {
        "contents": [
            {"parts": [{"text": f"Context:\n{context}\n\nUser Query: {query}"}]}
        ]
    }

    print("\n--- DEBUG: Gemini API Call ---")
    print("Endpoint:", endpoint)
    print("Headers:", headers)
    print("Payload:", payload)

    response = requests.post(f"{endpoint}?key={api_key}", headers=headers, json=payload)

    print("Response Status Code:", response.status_code)
    print("Response Content:", response.text)

    if response.status_code == 200:
        data = response.json()
        # Extract the response text from candidates
        try:
            return data["candidates"][0]["content"]["parts"][0]["text"]
        except (KeyError, IndexError) as e:
            print("Error parsing response:", str(e))
            print("Full response:", data)
            return "No response: Unable to parse the result."
    else:
        print("Gemini API Error:", response.text)
        return f"Error {response.status_code}: {response.text}"

# Step 6: Combine Retrieval and Generation
def rag_pipeline(query, api_key, top_k=7):
    retrieved_context = retrieve(query, top_k=top_k)
    if not retrieved_context.strip():
        print("No context retrieved.")  # Debugging log
        return "No relevant context found for your query."

    response = generate_with_gemini(retrieved_context, query, api_key)
    return response

# Flask API Setup
app = Flask(__name__)

@app.route("/recommend", methods=["POST"])
def recommend():
    data = request.json
    query = data.get("query")
    api_key = data.get("api_key")

    if not query or not api_key:
        return jsonify({"error": "Missing 'query' or 'api_key'"}), 400

    response = rag_pipeline(query, api_key, top_k=7)
    return jsonify({"response": response})

# Expose the Flask app via Ngrok
ngrok.set_auth_token("")  # Replace with your Ngrok token
public_url = ngrok.connect(5000).public_url
print(f"Public URL: {public_url}")

if __name__ == "__main__":
    app.run(port=5000)

Public URL: https://27db-34-168-137-99.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit



--- DEBUG: Retrieved Contexts ---
Exercise Name: Isometric Chest Squeezes
Equipment: body only
Primary Muscles: chest
Secondary Muscles: chest
Instructions: While either seating or standing, bend your arms at a 90-degree angle and place the palms of your hands together in front of your chest. Tip: Your hands should be open with the palms together and fingers facing forward (perpendicular to your torso). Push both hands against each other as you contract your chest. Start with slow tension and increase slowly. Keep breathing normally as you execute this contraction. Hold for the recommended number of seconds. Now release the tension slowly. Rest for the recommended amount of time and repeat..
Exercise Name: Chest Push from 3 point stance
Equipment: medicine ball
Primary Muscles: chest
Secondary Muscles: chest
Instructions: Begin in a three point stance, squatted down with your back flat and one hand on the ground. Place the medicine ball directly in front of you. To begin, take your fi

INFO:werkzeug:127.0.0.1 - - [15/Jan/2025 11:08:57] "POST /recommend HTTP/1.1" 200 -


Response Status Code: 200
Response Content: {
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "Here are 3 chest exercises from the provided text:\n\n1. **Isometric Chest Squeezes:** This bodyweight-only exercise involves squeezing your hands together in front of your chest to build tension and strengthen the pectoral muscles.\n\n2. **Decline Push-Up:** This exercise uses your body weight and requires elevating your feet to increase the emphasis on the upper chest.\n\n3. **Dips - Chest Version:**  This exercise uses parallel bars and focuses on the chest by leaning forward during the movement.\n"
          }
        ],
        "role": "model"
      },
      "finishReason": "STOP",
      "avgLogprobs": -0.16305928911481585
    }
  ],
  "usageMetadata": {
    "promptTokenCount": 847,
    "candidatesTokenCount": 105,
    "totalTokenCount": 952
  },
  "modelVersion": "gemini-1.5-flash-latest"
}


--- DEBUG: Retrieved Contexts ---
Exercise Name: 

INFO:werkzeug:127.0.0.1 - - [15/Jan/2025 11:11:17] "POST /recommend HTTP/1.1" 200 -


Response Status Code: 200
Response Content: {
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "Here are 3 leg exercises from the provided list, offering variety in muscle focus:\n\n1. **Alternate Leg Diagonal Bound:** This exercise focuses on the quadriceps and improves agility and lateral movement.  It requires no equipment.\n\n2. **Box Skip:** This exercise primarily works the hamstrings, providing a different leg muscle focus than the previous one. It requires several boxes.\n\n3. **Frog Hops:** This exercise targets the quadriceps and improves explosive power. No equipment is needed.\n"
          }
        ],
        "role": "model"
      },
      "finishReason": "STOP",
      "avgLogprobs": -0.16924757115981159
    }
  ],
  "usageMetadata": {
    "promptTokenCount": 774,
    "candidatesTokenCount": 102,
    "totalTokenCount": 876
  },
  "modelVersion": "gemini-1.5-flash-latest"
}


--- DEBUG: Retrieved Contexts ---
Exercise Name: Windm

INFO:werkzeug:127.0.0.1 - - [15/Jan/2025 11:11:54] "POST /recommend HTTP/1.1" 200 -


Response Status Code: 200
Response Content: {
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "Here are two exercises from the previous text:\n\n1. **Windmills:** Lie on your back with arms extended to the sides and legs straight.  Lift one leg and quickly cross it over your body, touching the ground near the opposite hand. Alternate legs for 10-20 repetitions.\n\n2. **Box Jump (Multiple Response):**  Facing a box, jump upward and forward, landing with both feet simultaneously on the box. Immediately jump back down and repeat.\n"
          }
        ],
        "role": "model"
      },
      "finishReason": "STOP",
      "avgLogprobs": -0.096659294282547145
    }
  ],
  "usageMetadata": {
    "promptTokenCount": 722,
    "candidatesTokenCount": 99,
    "totalTokenCount": 821
  },
  "modelVersion": "gemini-1.5-flash-latest"
}

